# Symbolically Assisted $\Lambda^*$ 

## Introduction

We present Symbolically Assisted $\Lambda^*$ (SA-$\Lambda^*$), a modification of $\Lambda^*$ that incorporates symbolic information retrieved via Coastal, our concolic execution tool.

$\Lambda^*$ consists of two components, a Learner and an Oracle. The 
Learner can issue two types of questions to the Oracle: membership queries and equivalence queries. The Oracle responds to membership queries with `true` (if the word $w$ is in the language $\mathcal{L}$ accepted by the automaton) or `false` otherwise. Equivalence queries return `true` if the hypothesis automaton is equivalent to the target automaton, otherwise a counter example is returned to the Learner.

The Learner updates an internal data structure called the _observation table_. In classic $\Lambda^*$, the row and column labels are concrete symbols from $\Sigma^*$. In SA-$\Lambda^*$, labels are constraints from $C^*$. For each $x \in \mathbb{W}$ and $p \in C$, $p(x) = \text{true} \vert \text{false}$. For now, we restrict ourselves to $W$, the domain of integers.

Once the observation table is closed and consistent, the Learner must produce an hypothesis automaton in order to issue an equivalence query. 

In classic $\Lambda^*$ this happens in two steps: the creation of an evidence automaton (a DFA), and the learning of the intervals along the transitions (resulting in a SFA). 

In SA-$\Lambda^*$, we leverage the path conditions collected when running the target parser symbolically via Coastal. From these path conditions, we can determine two things: the exact constraint on each character of the symbolic input at the end of the execution, and if this input resulted in the parser accepting or rejecting.

The hypothesis automaton is constructed from the observation table. SA-$\Lambda^*$ uses the same procedures as classic $\Lambda^*$. The semantics of appending constraints is straightforward: if $c_0, c_1 \in C$, and $w = w_0w_1 \in \Sigma^*$, then the constraint $c_0 \cdot c_1$ is satisfied _iff_ $c_0(w_0) = \text{true}$ and $c_1(w_1) = \text{true}$. Appending $\epsilon$ is a no-op, it does not append a new constraint.

**TODO** Simple explanation of the table operations.

## Prelude

In [1]:
from IPython.display import HTML, display
import tabulate
from tabledefs import *

def display_table(table):
    display(HTML(tabulate.tabulate(table, tablefmt='html')))

## Worked Example (TACAS2017)

This example is taken from _Learning Symbolic Automata_. The target parser  accepts all sequences of non-negative integers, _except_ sequences where there is a number $51 \le x \le 100$, that is not immediately followed by two numbers $y$, $z$ where $0 \le y, z \le 20$. 

We begin with out initial observation table. We sample an initial constraint from the set of symbolically collected constraints. $c_0 = [0\ 50]$, where $[n\ m]$ is notation the represent the constraint $n \le x \le m$.

In [2]:
display_table(table1)

𝑇1,𝜖
𝜖,+
,
c0,+


The table is closed and consistent, so we create the hypothesis automaton and issue and equivalence query.

![T1](t1.jpeg)

### Counter Example 1

The first counter example is $(c_1 = [51\ 100], -)$, where the first item in the tuple is the counter example, and the second item represents whether the parser accepted or rejected the input. New constraints are added to $C$ as they are discovered. Here, we have a new constraint that leads to a reject state. 

**TODO** Explain the state creation of close and the evidence expanding of make consistent

We process the counter example, adding it to the and all it's prefixes into $R$.

In [3]:
display_table(table2)

𝑇2,𝜖
𝜖,+
,
c0,+
c1,-


The Learner performs a close operation, introducing a fail state into the automaton. $c_1$ is moved from $R$ to $S$, and we are required to add add $c_1 \cdot a$. In classic $\Lambda^*$, this is an arbritary character from the alphabet $\Sigma$. In SF-$\Lambda^*$, we attempt to further leverage the symbolic information provided by Coastal. We execute the target parser symbolically with an input of length two, where the first constraint is fixed to $c1$. We then select an arbritary constraint from the path conditions returned. In this case, $c_2 = [0\ 20]$.

In [4]:
display_table(table3)

𝑇3,𝜖
𝜖,+
c1,-
,
c0,+
c1 ⋅ c2,-


![T3](t3.jpeg)

### Counter Example 2

The second counter example is $(c3 = [101\ \inf]), +)$. It is added to $R$.

In [5]:
display_table(table4)

𝑇4,𝜖
𝜖,+
c1,-
,
c0,+
c1 ⋅ c2,-
c3,+


![T4](t4.jpeg)

### Counter Example 3

The third counter example is $((c_1 \cdot c_2 \cdot c_2), +)$, where $c_1 = [51\ 100]$ and $c_2 = [0\ 20]$, and it is accepted by the target parser. We add the counter example and all its prefixes to $R$.

In [6]:
display_table(table5)

𝑇5,𝜖
𝜖,+
c1,-
,
c0,+
c1 ⋅ c2,-
c3,+
c1 ⋅ c2 ⋅ c2,+


We now apply `make-consistent` to the observation table.

In [7]:
display_table(table6)

𝑇6,𝜖,c2
𝜖,+,+
c1,-,-
,,
c0,+,+
c1 ⋅ c2,-,+
c3,+,+
c1 ⋅ c2 ⋅ c2,+,+


We now apply `close` to move $c_1 \cdot c_2$ into $S$.

In [8]:
display_table(table7)

𝑇7,𝜖,c2
𝜖,+,+
c1,-,-
c1 ⋅ c2,-,+
,,
c0,+,+
c3,+,+
c1 ⋅ c2 ⋅ c2,+,+


![T7](t7.jpeg)

### Counter Example 4

The fourth counter example is $((c_1 \cdot c_4), -)$, where $c_4 =\ ! c_3 = [21\ \inf]$.

In [9]:
display_table(table8)

𝑇8,𝜖,c2
𝜖,+,+
c1,-,-
c1 ⋅ c2,-,+
,,
c0,+,+
c3,+,+
c1 ⋅ c2 ⋅ c2,+,+
c1 ⋅ c4,-,-


![T8](t8.jpeg)

### Counter Example 5

The fifth counter example is $((c_1 \cdot c_4 \cdot c_3 \cdot c_3), -)$.

In [10]:
display_table(table9)

𝑇9,𝜖,c2
𝜖,+,+
c1,-,-
c1 ⋅ c2,-,+
,,
c0,+,+
c3,+,+
c1 ⋅ c2 ⋅ c2,+,+
c1 ⋅ c4,-,-
c1 ⋅ c4 ⋅ c3 ⋅ c3,-,-
c1 ⋅ c4 ⋅ c3,-,-


Apply `make-consistent` to the table.

In [11]:
display_table(table10)

𝑇10,𝜖,c2,c3 ⋅ c3
𝜖,+,+,+
c1,-,-,+
c1 ⋅ c2,-,+,+
,,,
c0,+,+,+
c3,+,+,+
c1 ⋅ c2 ⋅ c2,+,+,+
c1 ⋅ c4,-,-,-
c1 ⋅ c4 ⋅ c3 ⋅ c3,-,-,-
c1 ⋅ c4 ⋅ c3,-,-,-


Apply `close` to the table.

In [12]:
display_table(table11)

𝑇11,𝜖,c2,c3 ⋅ c3
𝜖,+,+,+
c1,-,-,+
c1 ⋅ c2,-,+,+
c1 ⋅ c4,-,-,-
,,,
c0,+,+,+
c3,+,+,+
c1 ⋅ c2 ⋅ c2,+,+,+
c1 ⋅ c4 ⋅ c3 ⋅ c3,-,-,-
c1 ⋅ c4 ⋅ c3,-,-,-


![T11](t11.jpeg)

## Broken Example (TACAS2017 FUBAR)

The target automaton has a similar structure to the original TACAS example, but with predicates changed to be less _convienient_, in the sense that they overlap partially or are totally disjoint.

![targetfub](targetfub.jpeg)

#### Comparison with `symbolicautomata`

For comparison sake, `symbolicautomata` is able to learn this.

![learnme](learnme.png)

---------------------------------

Constraint $(c_1 = [21\ \inf], +)$ is sampled from a set of constraint / acceptance pairs obtained from a symbolic execution of depth 1 via Coastal.

In [23]:
display_table(ftable1)

𝑇1,𝜖
𝜖,+
,
c1,+


### Counter Example 1

The first counter example is $(c_1 = [0\ 20], -)$.

In [14]:
display_table(ftable2)

𝑇2,𝜖
𝜖,+
,
c1,+
c2,-


Apply `close` to the table. Constraint $c_3 = [0\ 24]$ is sampled from a set of constraints collected by Coastal, where the first constraint in the path condition was $c_2$. 

In [15]:
display_table(ftable3)

𝑇3,𝜖
𝜖,+
c2,-
,
c1,+
c2⋅c3,-


### Counter Example 2

The second counter example is $(c_2 \cdot c_4, -)$, with $c_4 = [25\ 30]$.


In [16]:
display_table(ftable4)

𝑇4,𝜖
𝜖,+
c2,-
,
c1,+
c2⋅c3,-
c2⋅c4,-


### Counter Example 3

The third counter example is $(c_2 \cdot c_5, -)$, with $c_5 = [99\ 99]$.


In [17]:
display_table(ftable5)

𝑇5,𝜖
𝜖,+
c2,-
,
c1,+
c2⋅c3,-
c2⋅c4,-
c2⋅c5,-


### Counter Example 4

The fourth counter example is $(c_2 \cdot c_4 \cdot c_6, +)$, with $c_6 = [0\ 10]$.


In [18]:
display_table(ftable6)

𝑇6,𝜖
𝜖,+
c2,-
,
c1,+
c2⋅c3,-
c2⋅c4,-
c2⋅c5,-
c2⋅c4⋅c6,+


### Counter Example 5

The fifth counter example is $(c_2 \cdot c_5 \cdot c_6, -)$. 


In [22]:
display_table(ftable7)

𝑇7,𝜖
𝜖,+
c2,-
,
c1,+
c2⋅c3,-
c2⋅c4,-
c2⋅c5,-
c2⋅c4⋅c6,+
c2⋅c5⋅c6,-


Apply `make-consistent` to the table.

In [21]:
display_table(ftable8)

𝑇8,𝜖,c6
𝜖,+,
c2,-,
,,
c1,+,
c2⋅c3,-,
c2⋅c4,-,+
c2⋅c5,-,
c2⋅c4⋅c6,+,
c2⋅c5⋅c6,-,


Note how the only valid path condition is $c_2 \cdot c_4$. All other concatenations of constraints in $S \cup R$ with constraints in $E$.